#Redes Neuronales Convolucionales

Este tutorial es para raza bien pesada en machine learning :/

#Descripcion General

La clasificacion CIFAR-10 es un problema comun de benchmark en machine learning. El problema es clasificar las imagenes RGB de 32 X 32 pixeles en las siguientes 10 categorias: Avion, Carro, Pajaro, Gato, Venado, Perro, Rana, Caballo, Barco y Trailers.

![Image of Yaktocat](https://www.tensorflow.org/versions/master/images/cifar_samples.png)

#Objetivos

El objetivo de este tutorial es construir relativamente una pequena Red Neuronal Convolucional para reconocer imagenes. En el proceso de este tutorial:

1. Se resalta una organizacion canonical para la arquitectura de red, entrenamiento y evaluacion.
2. Provee un template para construir un modelo mas grande y sofisticado.

#Aspectos destacados del Tutorial

El tutorial CIFAR-10 demuestra varias construcciones importantes para un disenio grande y modelos mas sofisticados en TensorFlow:

* Componentes matematicos basicos incluidos tales como convolution, rectified linear activations, max pooling y local response normalization.
* Visualizacion de la red de actividades durante el entrenamiento incluyendo imagenes de entrada, perdidas y distribuciones de activacion y gradientes.
* Rutinas para calcular el moving average de los parametros de aprendisaje utilizando estos promedios durante la evaluacion para aumentar el rendimiento de prediccion.
* Implementacion de un learning rate schedule que sistematicamente decrese sobre el tiempo.
* Prefetching queues para los datos de entrada para isolar el modelo de la latencia del disco y el preprocesamiento de imagenes costosas.

#Arquitectura del Modelo

El modelo en este tutorial de CIFAR-10 es una arquitectura multicapas que consiste de convoluciones alternas y nonlinearities. Estas capas estan seguidas de capas fully connected dirigidas dentro del clasifiador softmax.

#Organizacion del Codigo

El codigo de este tutorial se encuentra en el github del proyecto.

Los archivos a utilizar son los siguientes:

* cifar10_input.py - Lee el formato nativo en binario del archivo CIFAR-10.
* cifar10.py - Construye el modelo CIFAR-10
* cifar10_train.py - Entrena el modelo CIFAR-10 en un CPU.
* cifar10_eval.py - Evalua el rendimiento de la exactitud de el Modelo CIFAR-10.


#Modelo CIFAR-10

La red CIFAR-10 esta contenida en gran parte en cifar10.py. El entrenamiento completo del grafo contiene aproximadamente 765 operaciones. El codigo se puede hacer mas reutilizable mediante la construccion en el grafo de los siguientes modulos:

1. Model inputs: inputs() y distorted_inputs() agrega operaciones que leen el preprocesamiento de las imagenes de CIFAR para la evaluacion y entrenamiento respectivamente.
2. Model prediction: inference() agrega operaciones que realizan inferencia por ejemplo clasificacion, en las imagenes dadas.
3. Model training: loss() y train() agrega operaciones que calculan la perdida, gradientes, actualizaciones de variables y visualizacion de recopilatorios.

#Entradas del Modelo

La parte de entrada del modelo es contruido por las funciones inputs() y distorted_inputs() lo cual leen imagenes de los archivos de datos binarios de CIFAR-10. Estos archivos contienen registros de bytes de longitud fija entonces se usa tf.FixedLengthRecordReader.

Las imagenes son procesadas de la siguiente manera:

* Las imagenes se recortan a un tamanio de 24 X 24 pixeles centralmente para la evaluacion aleatoria en el entrenamiento.
* Las imagenes estan aproximadamente whitened para hacer el modelo insensible en el rango dinamico.

Para entrenamiento adicionalmente se aplican series de distiorsiones aleatorias para incrementar artificialmente el tamanio del dataset:

* Se voltea aleatoriamente la imagen de izquierda a derecha.
* Se distorciona aleatoriamente el brillo de la imagen.
* Se distorciona aleatoriamente el contraste de la imagen.



#Prediccion del Modelo

La parte de la prediccion del modelo se contruye con la funcion de inference() la cual agrega operaciones para calcular los logits de las predicciones. Esta parte del modelo se organiza como sigue:

Nombre de la capa - Descripcion
* conv1             - Convolution y rectified linear activation.
* pool1             - max polling.
* norm1             - local response normalization.
* conv2             - convolution y rectified linear activation.
* norm2             - local response normalization.
* pool2             - max pooling.
* local3            - fully connected layer with rectified linear activation.
* local4            - fully connected layer with rectified linear activation.
* softmax_linear    - Transformacion lineal que produce logits.

Aqui se presenta el grafo el cual fue generado por TensorBoard el cual describe la operacion de inferencia:

![Image of Yaktocat](https://www.tensorflow.org/versions/master/images/cifar_graph.png)

Las funciones inputs() y inference() proveen todos los componentes necesarios para realizar la evaluacion de un modelo. Ahora se cambiara el enfoque hacia las operaciones de construccion para el entrenamiento de un modelo.


#Entrenamiento del Modelo

El metodo usual para entrenar una red es realizar la clasificacion N-Way la cual es una regresion logistica multinomial regresion softmax. La regresion Softmax aplica un softmax nonlinearity a la salida de la red y calcula la cross-entropy entre las predicciones normalizadas y un 1-hot codificado de la etiqueta. Para la regularizacion se aplica usualmente la perdida de weight decay para todas las variables de aprendisaje. La funcion objetivo para el modelo es la suma de perdida de cross-entropy y todos sus pesos en terminos de decadencia esto se regresa con la funcion loss().

Se puede visualizar la perdida total en el TensorBoard con un scalar_summary:

![Image of Yaktocat](https://www.tensorflow.org/versions/master/images/cifar_loss.png)

Se entrena el modelo usando el algoritmo de descenso de gradiente estandar con una taza de aprendisaje que decae exponencialmente sobre el tiempo.

![Image of Yaktocat](https://www.tensorflow.org/versions/master/images/cifar_lr_decay.png)

La funcion train() agrega las operaciones necesarias para minimizar el objetivo tal que calcula el gradiente y actualiza las variables de aprendisaje. Regresa una operacion que ejecuta todos los calculos necesarios para entrenar y actualizar el modelo para un batch de imagenes.

#Inicializando y Entrenando el Modelo

Ya que se construyo el modelo se pasa a inicializarlo y ejecutar la operacion de entrenamiento con el script cifar10_train.py.

In [ ]:
import os
filepath='/home/juanmanuel/Desktop/deep-learning/cifar10'
os.chdir(filepath)
%run cifar10_train.py

>> Downloading cifar-10-binary.tar.gz 20.3%